In [1]:
# 패키지 설치
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, precision_score, recall_score, auc
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# 데이터 불러오기
respec = pd.read_csv('/content/gdrive/MyDrive/BigContest_data/respec.csv')
respec

,application_id,user_id,gender,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,reage
0,311143,646289,1.0,2022-06-07 10:17:19,530.0,28000000.0,3,2022.0,계약직,기타가족소유,3000000.0,LIVING,0.0,0.0,5.0,19000000.0,20.0
1,1903409,168234,0.0,2022-06-07 09:43:20,560.0,12000000.0,5,2021.0,기타,기타가족소유,3000000.0,LIVING,0.0,0.0,10.0,23000000.0,30.0
2,1237618,717213,1.0,2022-06-07 03:26:54,530.0,51000000.0,3,2017.0,정규직,전월세,5000000.0,LIVING,0.0,0.0,0.0,0.0,30.0
3,647624,796103,1.0,2022-06-07 23:07:42,520.0,61000000.0,3,2019.0,정규직,전월세,5000000.0,LIVING,0.0,0.0,7.0,96000000.0,40.0
4,223654,325066,0.0,2022-06-07 16:39:55,570.0,26000000.0,3,2020.0,정규직,전월세,5000000.0,LIVING,0.0,0.0,5.0,58000000.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394126,1864587,489900,1.0,2022-03-22 14:55:32,590.0,25000000.0,2,2021.0,기타,기타가족소유,5000000.0,BUSINESS,0.0,0.0,0.0,0.0,20.0
1394127,1327066,151422,1.0,2022-03-22 01:19:24,980.0,20000000.0,4,2017.0,기타,자가,50000000.0,LIVING,1.0,1.0,1.0,31000000.0,60.0
1394128,1319606,173524,1.0,2022-03-22 07:34:32,750.0,75000000.0,3,2009.0,정규직,자가,100000000.0,SWITCHLOAN,1.0,1.0,8.0,200000000.0,40.0
1394129,1482466,766546,1.0,2022-03-22 22:12:35,640.0,50000000.0,3,2017.0,정규직,자가,10000000.0,SWITCHLOAN,0.0,1.0,10.0,117000000.0,40.0


In [4]:
respec = respec.drop("application_id", axis=1)

In [5]:
respec = respec.drop("user_id", axis=1)

In [6]:
respec = respec.drop("insert_time", axis=1)

In [7]:
respec_test = respec[respec['credit_score'].isnull()]
respec_test

,gender,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,reage
8,0.0,NaN,28000000.0,3,2013.0,정규직,전월세,10000000.0,LIVING,0.0,0.0,3.0,52000000.0,30.0
15,1.0,NaN,45000000.0,3,2016.0,정규직,전월세,18000000.0,SWITCHLOAN,0.0,0.0,0.0,0.0,30.0
22,1.0,NaN,35000000.0,3,2021.0,정규직,기타가족소유,3000000.0,LIVING,0.0,0.0,0.0,0.0,20.0
23,1.0,NaN,11000000.0,3,2021.0,정규직,전월세,1000000.0,LIVING,0.0,0.0,0.0,0.0,30.0
24,0.0,NaN,35000000.0,3,2018.0,정규직,전월세,100000000.0,HOUSEDEPOSIT,0.0,0.0,0.0,0.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394035,1.0,NaN,100000000.0,1,2021.0,정규직,기타가족소유,50000000.0,BUSINESS,0.0,0.0,0.0,0.0,30.0
1394042,1.0,NaN,30000000.0,3,2018.0,정규직,전월세,10000000.0,LIVING,0.0,0.0,0.0,0.0,30.0
1394056,1.0,NaN,44000000.0,3,2018.0,정규직,전월세,20000000.0,ETC,1.0,0.0,0.0,0.0,30.0
1394094,1.0,NaN,10000000.0,2,2021.0,기타,기타가족소유,3000000.0,HOUSEDEPOSIT,1.0,0.0,0.0,0.0,20.0


In [8]:
respec_train = respec[respec['credit_score'].notnull()]
respec_train

,gender,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,reage
0,1.0,530.0,28000000.0,3,2022.0,계약직,기타가족소유,3000000.0,LIVING,0.0,0.0,5.0,19000000.0,20.0
1,0.0,560.0,12000000.0,5,2021.0,기타,기타가족소유,3000000.0,LIVING,0.0,0.0,10.0,23000000.0,30.0
2,1.0,530.0,51000000.0,3,2017.0,정규직,전월세,5000000.0,LIVING,0.0,0.0,0.0,0.0,30.0
3,1.0,520.0,61000000.0,3,2019.0,정규직,전월세,5000000.0,LIVING,0.0,0.0,7.0,96000000.0,40.0
4,0.0,570.0,26000000.0,3,2020.0,정규직,전월세,5000000.0,LIVING,0.0,0.0,5.0,58000000.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394125,1.0,730.0,60000000.0,1,2021.0,정규직,전월세,15000000.0,BUSINESS,0.0,1.0,1.0,31000000.0,20.0
1394126,1.0,590.0,25000000.0,2,2021.0,기타,기타가족소유,5000000.0,BUSINESS,0.0,0.0,0.0,0.0,20.0
1394127,1.0,980.0,20000000.0,4,2017.0,기타,자가,50000000.0,LIVING,1.0,1.0,1.0,31000000.0,60.0
1394128,1.0,750.0,75000000.0,3,2009.0,정규직,자가,100000000.0,SWITCHLOAN,1.0,1.0,8.0,200000000.0,40.0


In [9]:
# 인코딩
respec_train = pd.get_dummies(respec_train)
respec_test= pd.get_dummies(respec_test)

In [10]:
data = respec_train.drop('credit_score',axis=1)
target = respec_train['credit_score']

In [11]:
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
print(standardScaler.fit(data))
data_scaled = standardScaler.transform(data)

StandardScaler()


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data_scaled, target, test_size=0.3, random_state=0)

# LightGBM

In [13]:
from lightgbm import LGBMRegressor

In [14]:
lgb = LGBMRegressor()
lgb.fit(X_train, y_train)
lgb_preds= lgb.predict(X_val)

In [15]:
lgb_preds

array([565.93754283, 393.92896381, 656.95329419, ..., 725.835796  ,
       673.15999252, 687.01214852])

In [16]:
mse = mean_squared_error(y_val, lgb_preds)
rmse = np.sqrt(mse)

print('MSE : {0:.3f}, RMSE : {1:.3F}'.format(mse , rmse))
print('Variance score : {0:.3f}'.format(r2_score(y_val,lgb_preds)))

MSE : 7687.389, RMSE : 87.678
Variance score : 0.508


# LightGBM- 파라미터 튜닝

In [17]:
from sklearn.metrics import fbeta_score, make_scorer

In [18]:
def score_mse(test, predict):
  return mean_squared_error(test, predict)

In [19]:
from sklearn.model_selection import GridSearchCV

params = {
    'num_leaves': [7, 14, 21, 28],
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5],
    'n_estimators': [50, 100],
}

grid = GridSearchCV(LGBMRegressor(random_state=0), params, scoring= make_scorer(score_mse), cv=5)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LGBMRegressor(random_state=0),
             param_grid={'learning_rate': [0.1, 0.01], 'max_depth': [3, 5],
                         'n_estimators': [50, 100],
                         'num_leaves': [7, 14, 21, 28]},
             scoring=make_scorer(score_mse))

In [20]:
print('최적 하이퍼 파라미터:{0}, 최적 mse:{1:.3f}'.format(grid.best_params_,
                                                 grid.best_score_))

최적 하이퍼 파라미터:{'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'num_leaves': 7}, 최적 mse:11515.887


## 몇가지 파라미터 그리드서치로 바꿔가면서 돌려봤는데 기본 보다 더 좋게 나오는건 못 찾음..